In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import pickle
import random
import time
import math
import numpy as np
from konlpy.tag import Mecab;tagger=Mecab()
from collections import Counter
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

torch.manual_seed(1)

In [2]:
USE_CUDA = torch.cuda.is_available()

In [3]:
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w], seq))
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

In [ ]:
MODEL_DIM = 512 # d_model = d_k = d_v
N = 6 # num of layer


In [6]:
1.0e4==10000

True

In [ ]:
math.lo

In [7]:
def add_timing_signal_1d(x, min_timescale=1.0, max_timescale=1.0e4):
  """Adds a bunch of sinusoids of different frequencies to a Tensor.
  Each channel of the input Tensor is incremented by a sinusoid of a different
  frequency and phase.
  This allows attention to learn to use absolute and relative positions.
  Timing signals should be added to some precursors of both the query and the
  memory inputs to attention.
  The use of relative position is possible because sin(x+y) and cos(x+y) can be
  experessed in terms of y, sin(x) and cos(x).
  In particular, we use a geometric sequence of timescales starting with
  min_timescale and ending with max_timescale.  The number of different
  timescales is equal to channels / 2. For each timescale, we
  generate the two sinusoidal signals sin(timestep/timescale) and
  cos(timestep/timescale).  All of these sinusoids are concatenated in
  the channels dimension.
  Args:
    x: a Tensor with shape [batch, length, channels]
    min_timescale: a float
    max_timescale: a float
  Returns:
    a Tensor the same shape as x.
  """
  length = tf.shape(x)[1]
  channels = tf.shape(x)[2]
  position = tf.to_float(tf.range(length))
  num_timescales = channels // 2
  log_timescale_increment = (
      math.log(float(max_timescale) / float(min_timescale)) /
      (tf.to_float(num_timescales) - 1))
  inv_timescales = min_timescale * tf.exp(
      tf.to_float(tf.range(num_timescales)) * -log_timescale_increment)
  scaled_time = tf.expand_dims(position, 1) * tf.expand_dims(inv_timescales, 0)
  signal = tf.concat([tf.sin(scaled_time), tf.cos(scaled_time)], axis=1)
  signal = tf.pad(signal, [[0, 0], [0, tf.mod(channels, 2)]])
  signal = tf.reshape(signal, [1, length, channels])
  return x + signal

In [9]:
torch.mean(torch.from_numpy(np.array([1.0,2.0])))

1.5

In [13]:
class Transformer(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=6):
        super(Transformer, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(vocab_size, hidden_size)

    
    def positional_embedding(self, x, min_timescale=1.0, max_timescale=1.0e4):
        """
          Args:
            x: a Tensor with shape [batch, length, channels]
            min_timescale: a float
            max_timescale: a float
          Returns:
            a Tensor the same shape as x.
        """
        return x
    
    def layer_norm(self, a, gain,bias):
        """
        a = Batch_size*Model_dim
        g = gain_parameter
        """
        mu = torch.mean(a, dim=1)
        sigma = torch.std(a, dim=1)
        
        return (gain/sigma)*(a-mu)+bias
    
    def residual(self,l_input,l_output):
        return self.layer_norm(l_input + F.dropout(l_output,0.1))
    
    
    def dot_attention(self,query, kvpairs):
        """
        Scaled Dot-Product Attention
        """
        
        hidden = hidden.squeeze(0).unsqueeze(2)  # 히든 : (1,배치,차원) -> (배치,차원,1)
        
        batch_size = encoder_outputs.size(0) # B
        max_len = encoder_outputs.size(1) # T
        energies = self.attn(encoder_outputs.contiguous().view(batch_size*max_len,-1)) # B*T,D -> B*T,D
        energies = energies.view(batch_size,max_len,-1) # B,T,D (배치,타임,차원)
        
        attn_energies = energies.bmm(hidden).transpose(1,2) # B,T,D * B,D,1 --> B,1,T 
        
        alpha = F.softmax(attn_energies.squeeze(1)) # B,T
        alpha = alpha.unsqueeze(1) # B,1,T
        context = alpha.bmm(encoder_outputs) # B,1,T * B,T,D => B,1,D
        
        
        return context # B,1,D
    
    def multihead_attention(self,something):
        pass
    
    
    def forward(self, input):
       
        embedded = self.embedding(input) # B,T,D
        embedded = self.positional_embedding(embedded) # positional embedding
        
        for n in range(len(self.n_layers)):
            output = self.multihead_attention(embedded)
            output = self.residual(embedded,output)
            
            
        return output, hidden